# Context Manager
- A context manager in Python is responsible for allocating and releasing resources precisely at the desired time. 
- It ensures that resources are set up when needed and then automatically cleaned up or released after use, even if an error occurs. 

### With

In [1]:
file = open("./file1.txt", 'w')
try:
    file.write("Test1.")
finally:
    file.close()

In [2]:
import os
print(os.listdir())

['.gitignore', '.ipynb_checkpoints', 'context_manager.ipynb', 'copy.ipynb', 'file1.txt', 'functions.ipynb', 'method_overloading.ipynb', 'method_overriding.ipynb', 'property.ipynb', 'scope.ipynb']


In [3]:
with open("./file2.txt", 'w') as f:
    f.write("Test2.")

In [4]:
print(os.listdir())

['.gitignore', '.ipynb_checkpoints', 'context_manager.ipynb', 'copy.ipynb', 'file1.txt', 'file2.txt', 'functions.ipynb', 'method_overloading.ipynb', 'method_overriding.ipynb', 'property.ipynb', 'scope.ipynb']


### `__enter__`, `__exit__`

In [7]:
# Context manager with exception handling

class FileWriter():
    def __init__(self, file_name, method):
        print("FileWriter started : __init__")
        self.file_obj = open(file_name, method)
        
    def __enter__(self):
        print("FileWriter started : __enter__")
        return self.file_obj
    
    def __exit__(self, exc_type, value, trace_back):
        print("FileWriter started : __exit__")
        if exc_type:
            print("Logging exxception {}".format((exc_type, value, trace_back)))
        self.file_obj.close()

In [8]:
# with -> init -> enter -> exit
with FileWriter('./file3.txt', 'w') as f:
    f.write("Test3.")

FileWriter started : __init__
FileWriter started : __enter__
FileWriter started : __exit__


In [10]:
# Contextlib: Measure execution

import time

class ExecuteTimer(object):
    def __init__(self, msg):
        self._msg = msg
        
    def __enter__(self):
        self._start = time.monotonic()
        return self._start
    
    # exit from `with`
    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type:
            print("Logging exception {}".format((exc_type, exc_value, exc_traceback)))
        else:
            print("{} : {} s".format(self._msg, time.monotonic() - self._start))
        return True

- v: returned value of the `__enter__`

In [11]:
with ExecuteTimer("Start...") as v:
    print("Received start monotonic1 : {}".format(v))
    # Execute job
    for i in range(1000000):
        pass

Received start monotonic1 : 1653231.328
Start... : 0.01500000013038516 s


In [12]:
with ExecuteTimer("Start...") as v:
    print("Received start monotonic1 : {}".format(v))
    # Execute job
    for i in range(1000000):
        pass
    raise Exception("Raise...")

Received start monotonic1 : 1653243.078
Logging exception (<class 'Exception'>, Exception('Raise...'), <traceback object at 0x0000016057945940>)


### `__enter__`, `__exit__`, `@contextlib.contextmanager`

In [13]:
import contextlib
import time

# Same with `ExecuteTimer` class
@contextlib.contextmanager
def file_writer(file_name, method):
    f = open(file_name, method)
    yield f  # __enter__
    f.close()  # __exit__
    
with file_writer("file4.txt", 'w') as f:
    f.write("Test4")

In [14]:
print(os.listdir())

['.gitignore', '.ipynb_checkpoints', 'context_manager.ipynb', 'copy.ipynb', 'file1.txt', 'file2.txt', 'file3.txt', 'file4.txt', 'functions.ipynb', 'method_overloading.ipynb', 'method_overriding.ipynb', 'property.ipynb', 'scope.ipynb']


In [15]:
# User decorator

@contextlib.contextmanager
def ExecuteTimer_2(msg):
    start = time.monotonic()
    try:  # __enter__
        yield start
    except BaseException as e:
        print("Logging exception: {}: {}".format(msg, e))
        raise
    else:  # __exit__
        print("{} : {}s".format(msg, time.monotonic() - start))
        

with ExecuteTimer_2("Start...") as v:
    print("Received start monotonic2 : {}".format(v))
    for i in range(100):
        pass
#     raise ValueError("raise...")

Received start monotonic2 : 1653286.031
Start... : 0.0s
